<a href="https://colab.research.google.com/github/rahinic/nlp-pos-bioes-tagging/blob/master/compositionNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Dictionary and Dirty Dataset acquisition:-


In [1]:
class compositionNetworksTagSets():

    def dictionary():
        # compose dictionary of possible sentence composition tags
        composition_tags = ['NP', 'PP', 'VP', 'ADVP', 'SBAR', 'ADJP', 'ADVP', 'PRT', 'CONJP', 'INTJ', 'LST', 'UCP', 'PADDING', 'UNAVAILABLE']
        composition_tags_lkp, composition_tags_lkp_rev = {}, {}
        
        for idx, tag in enumerate(composition_tags):
            composition_tags_lkp[tag] = idx
            composition_tags_lkp_rev[idx] = tag
        
        print("Dictionary ready!")
        return composition_tags_lkp, composition_tags_lkp_rev

class compositionNetworksDataset():
    # fetch datasets
    def load_corpus(self,filename):
        filepath = "/content/"
        f = open(filepath+str(filename),"r")
        content = f.readlines()

        return content

    def fetch_corpus(self,corpus_type):

        print(f"loading the {corpus_type} dataset")
        import sys

        types = ['train','test','validation']
        try:
            if corpus_type not in types:
                raise ValueError
        except ValueError as error:
            print('Invalid dataset type: possible values: "train/test/validation"')
            sys.exit(1)

        if corpus_type == "train":
            dataset = self.load_corpus(filename="02-21.10way.clean.txt")
        elif corpus_type == "test":
            dataset = self.load_corpus(filename="23.auto.clean.txt")
        elif corpus_type == "validation":
            dataset = self.load_corpus(filename="22.auto.clean.txt")

        return dataset

    def prepare_sentences(self,corpus_type):

        corpus_dataset = self.fetch_corpus(corpus_type)
        
        

        #import dict
        dict, dict_rev = compositionNetworksTagSets.dictionary()
        list_of_tags = list(dict.keys())

        return corpus_dataset


    def identify_phrases_size_n(self, find_chunks_from, chunk_type, chunk_size):

    #collects NP phrases of size 2 and 1 (at the end of sentence)
        list_of_phrases = []
        n = chunk_size 
        for sentence in find_chunks_from: # iterate through all sentences
            curr_sentence = sentence.split(' ')
            
            for idx, item in enumerate(curr_sentence): 

                # look if NP phrase exist in this sentence            
                if item == chunk_type:
                    NP_item_position = idx
                    if '))' in curr_sentence[idx:idx+n][-1]:
                        curr_phrase = curr_sentence[idx:idx+n] # n= 5
                        
                    else: continue #skipping chunks longer than 2 words
                    
                    # remove the end of sentence indicator:
                    if curr_phrase[-1] == '.)))\n':
                        curr_phrase =  curr_phrase[:-2]

                    list_of_phrases.append(tuple(curr_phrase))             
                
                # elif item != chunk_type: # skip until the next NP phrase in the same sentence
                #     continue
        return list_of_phrases

ds = compositionNetworksDataset()
dirty_sentences = ds.prepare_sentences(corpus_type="validation")
all_chunk_types = ['(NP', '(VP', '(PP','(SBAR','(ADJP','(PRT','(ADVP','(INTJ','(CONJP','(LST','(UCP','(PADDING','(UNAVAILABLE']

all_phrases = []
for type in all_chunk_types:
    for size in range(3,10,2):
        all_phrases.append(list(set(ds.identify_phrases_size_n(dirty_sentences, 
                                                            chunk_type = type, 
                                                            chunk_size=size)))) # NP phrases
  
all_phrases_final = list(filter(None, all_phrases))
#############################################################################

        



loading the validation dataset
Dictionary ready!


In [2]:

# compositionNetworksTagSets = compositionNetworksTagSets()
# compositionNetworksDataset 
import pickle, gzip
from typing import Tuple, List
import torch
from torch.utils.data import Dataset,DataLoader

class composeDataset(Dataset):

    # Step 1: look-up tables:
    def fetchLookupTables(self):
        dict = compositionNetworksTagSets
        composition_tags, composition_tags_reverse = dict.dictionary()

        return composition_tags, composition_tags_reverse

    def fetchWordsAndPOSLookupTables(self):
        def load_pickles(filename):
            data_raw_filepath = "/content/"

            if "pklz" in filename:     
                file = gzip.open(data_raw_filepath+str(filename),"rb")
            else:
                # print(f"loading: {data_raw_filepath+str(filename)}")
                file = open(data_raw_filepath+str(filename),"rb")

            return pickle.load(file)

        list_of_lookup_tables = ["nn1_wsj_idx_to_pos.pkl",
                         "nn1_wsj_pos_to_idx.pkl",
                         "nn1_wsj_word_to_idx.pkl",
                         "nn1_wsj_idx_to_word.pkl"]
        # (a) Vocabulory: 
        nn1_pos = load_pickles(list_of_lookup_tables[0])
        nn1_pos_reverse = load_pickles(list_of_lookup_tables[1])
        nn1_vocab = load_pickles(list_of_lookup_tables[2])
        nn1_vocab_reverse = load_pickles(list_of_lookup_tables[3])

        return nn1_vocab, nn1_pos, nn1_vocab_reverse, nn1_pos_reverse
        
    # Step 2: fetch dirty dataset
    def fetchDirtyDataset(self, dataset):
        ds = compositionNetworksDataset()
        dirty_sentences = ds.prepare_sentences(corpus_type=dataset)
        all_chunk_types = ['(NP', '(VP', '(PP','(SBAR','(ADJP','(PRT','(ADVP','(INTJ','(CONJP','(LST','(UCP','(PADDING','(UNAVAILABLE']

        all_phrases = []
        for type in all_chunk_types:
            for size in range(3,10,2):
                all_phrases.append(list(set(ds.identify_phrases_size_n(dirty_sentences, 
                                                                    chunk_type = type, 
                                                                    chunk_size=size)))) # NP phrases
        
        all_phrases_final = list(filter(None, all_phrases))
        return all_phrases_final

    # Step 3: Create tuples of (label, sentence) in a neat way:

    def cleanDirtyDataset(self,phrases):
        sentences_clean = []
        
        for phrase in phrases:
            
            sentence_pretty = []

            label = str(phrase[0]).replace('(','')
            
            sentence_ugly = list(phrase[1:])
            
            
            for item in sentence_ugly:
                token_less_ugly = str(item).replace('(','')
                token_less_ugly = token_less_ugly.replace(')','')
                token_less_ugly = token_less_ugly.replace('\n','')
                sentence_pretty.append(token_less_ugly)
            
            sentences_clean.append((label,sentence_pretty)) #tuple(label, sentence)

        return sentences_clean
            
    # Step 4: Convert tokens to idx:
    def prepareTensors(self,phrases):

        # pipelines
        def sample_word_pipeline(x):                        # word to idx
            return [self.penn_vocab[tok] for tok in x]
            
        def sample_pos_pipeline(x):                         # pos to idx
            return [self.penn_pos_tags[pos] for pos in x]



        samples = [item for sublist in phrases for item in sublist]
        all_samples = []
        skipped = 0
        for idx, sample in enumerate(samples):

            if len(sample)>20:
                continue
            
            label = sample[0]
            # print(sample[1])

            sentence, pos_tags = [], []

            #split sentence and pos tags
            for i in range(0,len(sample[1]),2):

                try:
                    pos_tags.append(sample[1][i])
                    sentence.append(sample[1][i+1])
                except IndexError:
                    continue

            if len(sentence)<20:
                for i in range(0,20-len(sentence)):
                    sentence.append('PADDING')
                    pos_tags.append('PADDING')
            
            # token to idx conversion
            
            try:
                sentence_to_idx = sample_word_pipeline(sentence)
                pos_to_idx = sample_pos_pipeline(pos_tags)
                label_to_idx = self.composition_tags[label]
            except KeyError:
                skipped = skipped+1
                # print(f"skipped {skipped} lines so far:")
                continue

            final_sample = [a + b for a, b in zip(sentence_to_idx, pos_to_idx)]

            sample_as_tensor = torch.tensor(final_sample)
            label_as_tensor = torch.tensor(label_to_idx)

            all_samples.append((label_as_tensor,sample_as_tensor))

        return all_samples

     
############################################################################################################

    def __init__(self, myDataset=None):

        self.myDataset = myDataset
        # print(self.myDataset)

        # Step 1: look-up tables:
        print("Loading Penn Treebank look-up tables...")
        self.composition_tags, self.composition_tags_reverse = self.fetchLookupTables()
        self.penn_vocab, _, _, self.penn_pos_tags = self.fetchWordsAndPOSLookupTables()
        
        print("done!")

        # Step 2: Fetching Pre-processed dataset:
        print("Pre-processing Penn Treebank dataset...")
        self.cleanedRawDataset = self.fetchDirtyDataset(dataset=self.myDataset)
        self.prettyDataset = []
        for dataset in self.cleanedRawDataset:
            self.prettyDataset.append(self.cleanDirtyDataset(dataset))
        print("done!")

        # Step 3: Convert the dataset to tensors
        print("Transforming the dataset to tensors...")
        self.final_dataset = self.prepareTensors(self.prettyDataset)
        print("done!")

    def __len__(self):

        return len(self.final_dataset)

    def __getitem__(self, idx) :

        return self.final_dataset[idx]
            

    
###################################################################




In [50]:
from torch import nn
import torch

"""RNN Many-to-one multi-class classification neural network model framework design"""

class RNNCompositionNetwork(nn.Module):

    def __init__(self, 
                embedding_dimension, 
                vocabulary_size,
                hidden_dimension,
                num_of_layers,
                dropout,
                output_dimension
                ):
        super(RNNCompositionNetwork, self).__init__()

        self.embedding = nn.Embedding(num_embeddings=vocabulary_size,
                                    embedding_dim=embedding_dimension)

        self.lstm = nn.LSTM(embedding_dimension,
                            hidden_dimension,
                            num_of_layers,
                            dropout=dropout,
                            batch_first=True)

        self.fc = nn.Linear(hidden_dimension*2, output_dimension)

        self.activation_fn = nn.ReLU()


    def forward(self, sample):
        # print(sample.size())
        embedded = self.embedding(sample)
        # print(embedded.size())
        output, (hidden, cell) = self.lstm(embedded)
        # print(hidden.size())
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-1,:,:], hidden[0,:,:]), dim = 1)
        # print(hidden.size())

        dense_output = self.fc(hidden)

        #activation function
        outputs=self.activation_fn(dense_output)

        return dense_output

In [61]:
from torch.utils.data import DataLoader
from torch import nn
import torch.optim as optim
import torch
# from dataset import composeDataset
composeDataset = composeDataset
# from dictionary import compositionNetworksTagSets
# from model import RNNCompositionNetwork
import pickle, time
##########################################################################################
################################# 01.TRain/Test Dataset ##################################

train_dataset = DataLoader(dataset=composeDataset(myDataset="train"),
                                batch_size=16,
                                shuffle=True)
                                
test_dataset = DataLoader(dataset=composeDataset(myDataset="test"),
                                batch_size=16,
                                shuffle=True)
##########################################################################################
################################ 02. Look-up tables ######################################
# (a) targets:-
dict1 = compositionNetworksTagSets
composition_tags, composition_tags_reverse = dict1.dictionary()
# (b) samples and pos tags:-
def fetchWordsAndPOSLookupTables():
    def load_pickles(filename):
        data_raw_filepath = "/content/"

        if "pklz" in filename:     
            file = gzip.open(data_raw_filepath+str(filename),"rb")
        else:
               # print(f"loading: {data_raw_filepath+str(filename)}")
            file = open(data_raw_filepath+str(filename),"rb")

        return pickle.load(file)

    list_of_lookup_tables = ["nn1_wsj_idx_to_pos.pkl",
                         "nn1_wsj_pos_to_idx.pkl",
                         "nn1_wsj_word_to_idx.pkl",
                         "nn1_wsj_idx_to_word.pkl"]
        # (a) Vocabulory: 
    nn1_pos_reverse = load_pickles(list_of_lookup_tables[0])
    nn1_pos = load_pickles(list_of_lookup_tables[1])
    nn1_vocab = load_pickles(list_of_lookup_tables[2])
    nn1_vocab_reverse = load_pickles(list_of_lookup_tables[3])

    return nn1_vocab, nn1_pos, nn1_vocab_reverse, nn1_pos_reverse


penn_vocab, _, _, penn_pos_tags = fetchWordsAndPOSLookupTables()

##########################################################################################
################################# 02.Model Parameters ####################################
VOCAB_SIZE = len(penn_vocab)+len(penn_pos_tags)+1
EMBED_DIM = 100
HIDDEN_DIM = 64
NUM_LAYERS = 1
NUM_OF_CLASSES = len(composition_tags)
EPOCHS = 10
LEARNING_RATE = 0.02
BATCH_SIZE = 64
################################### 02. NN Model  #######################################
print("03. builing the model...")
model = RNNCompositionNetwork(embedding_dimension= EMBED_DIM,
                            vocabulary_size=VOCAB_SIZE,
                            hidden_dimension=HIDDEN_DIM,
                            num_of_layers=NUM_LAYERS,
                            dropout=0,
                            output_dimension=NUM_OF_CLASSES)
print("----------------------------------------------------------------")
print("Done! here is our model:")
print(model)
print("----------------------------------------------------------------")
##########################################################################################
############################# 03. Optimizer and Loss  #################################
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
# optimizer = optim.Adam(model.parameters())
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    # correct = (rounded_preds == y).float() 
    _,pred_label = torch.max(rounded_preds, dim = 1)

    # sanity check:
    # print(f"Actual label: {y}")
    # print(rounded_preds)
    # print(f"Predicted label: {pred_label}")
    correct = (pred_label == y).float()
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
# model = model.to(device)
# criterion = criterion.to(device)

##########################################################################################
############################## 04. NN Model Train Definition #############################

def train(model, dataset, optimizer, criterion):
    
    t = time.localtime()
    start_time = time.strftime("%H:%M:%S", t)
    print(start_time)

    epoch_loss = 0
    epoch_accuracy = 0

    model.train()

    for idx, sample in enumerate(dataset):
       
       current_samples = sample[1]
       current_labels = sample[0]
    #    print(current_samples[:5])
       optimizer.zero_grad()

       predicted_labels = model(current_samples)
      #  print(predicted_label)

       
       loss = criterion(predicted_labels, current_labels)
       accuracy = binary_accuracy(predicted_labels, current_labels)

       loss.backward()
       optimizer.step()

       epoch_loss += loss.item()
       epoch_accuracy += accuracy.item()

    return epoch_loss/len(dataset), epoch_accuracy/len(dataset)

##########################################################################################
################################ 05. NN Model Eval Definition ############################
def evaluate(model, dataset, criterion):
    
    t = time.localtime()
    start_time = time.strftime("%H:%M:%S", t)
    print(start_time)

    epoch_loss = 0
    epoch_accuracy = 0
    model.eval()

    with torch.no_grad():

        for idx, sample in enumerate(dataset):
            current_samples = sample[1]
            current_labels = sample[0]

            predicted_labels = model(current_samples)

            loss = criterion(predicted_labels, current_labels)
            accuracy = binary_accuracy(predicted_labels, current_labels)

            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()

    return epoch_loss/len(dataset), epoch_accuracy/len(dataset)

############################################################################################
################################## 06. NN Model training #####################################
# EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(EPOCHS):

    print(epoch)
     
    #train the model
    train_loss, train_acc = train(model, train_dataset, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, test_dataset, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print("-------------------------------------------------------------------")
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print("-------------------------------------------------------------------")
    

Loading Penn Treebank look-up tables...
Dictionary ready!
done!
Pre-processing Penn Treebank dataset...
loading the train dataset
Dictionary ready!
done!
Transforming the dataset to tensors...
done!
Loading Penn Treebank look-up tables...
Dictionary ready!
done!
Pre-processing Penn Treebank dataset...
loading the test dataset
Dictionary ready!
done!
Transforming the dataset to tensors...
done!
Dictionary ready!
03. builing the model...
----------------------------------------------------------------
Done! here is our model:
RNNCompositionNetwork(
  (embedding): Embedding(46396, 100)
  (lstm): LSTM(100, 64, batch_first=True)
  (fc): Linear(in_features=128, out_features=13, bias=True)
  (activation_fn): ReLU()
)
----------------------------------------------------------------
0
08:59:59
09:03:03
-------------------------------------------------------------------
	Train Loss: 0.622 | Train Acc: 83.94%
	 Val. Loss: 0.628 |  Val. Acc: 82.13%
-------------------------------------------------

# Predictions and Model Accuracy Calculations:-

In [63]:
# 1. fetch dataset and few examples
validation_dataset = DataLoader(dataset=composeDataset(myDataset="validation"),
                                batch_size=16,
                                shuffle=True)

for idx, sample in enumerate(validation_dataset):
  if idx>1:
    break

  examples = sample[1][:2]
  actual_labels = sample[0][:2]
  
# 2. Model initialization and load weights

VOCAB_SIZE = len(penn_vocab)+len(penn_pos_tags)+1
EMBED_DIM = 100
HIDDEN_DIM = 64
NUM_LAYERS = 1
NUM_OF_CLASSES = len(composition_tags)
EPOCHS = 10
LEARNING_RATE = 0.02
BATCH_SIZE = 64
################################### 02. NN Model  #######################################
print("03. builing the model...")
model = RNNCompositionNetwork(embedding_dimension= EMBED_DIM,
                            vocabulary_size=VOCAB_SIZE,
                            hidden_dimension=HIDDEN_DIM,
                            num_of_layers=NUM_LAYERS,
                            dropout=0.2,
                            output_dimension=NUM_OF_CLASSES)
print("----------------------------------------------------------------")
print("Done! here is our model:")
print(model)
print("----------------------------------------------------------------")

# 3. Load weights
model.load_state_dict(torch.load("/content/saved_weights.pt"))
model.eval()




Loading Penn Treebank look-up tables...
Dictionary ready!
done!
Pre-processing Penn Treebank dataset...
loading the validation dataset
Dictionary ready!
done!
Transforming the dataset to tensors...
done!
03. builing the model...
----------------------------------------------------------------
Done! here is our model:
RNNCompositionNetwork(
  (embedding): Embedding(46396, 100)
  (lstm): LSTM(100, 64, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=13, bias=True)
  (activation_fn): ReLU()
)
----------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


RNNCompositionNetwork(
  (embedding): Embedding(46396, 100)
  (lstm): LSTM(100, 64, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=13, bias=True)
  (activation_fn): ReLU()
)

In [64]:

# 4. Make prediction
print(examples)
print(actual_labels)
with torch.no_grad():
  output = model(examples)
  
  rounded_preds = torch.round(output)
  
  _,pred_label = torch.max(rounded_preds, dim = 1)
  
  print(f"Actual labels: {actual_labels}")
  print(f"Predicted labels: {pred_label}")

tensor([[22867, 35225, 46393, 46393, 46393, 46393, 46393, 46393, 46393, 46393,
         46393, 46393, 46393, 46393, 46393, 46393, 46393, 46393, 46393, 46393],
        [29180, 46393, 46393, 46393, 46393, 46393, 46393, 46393, 46393, 46393,
         46393, 46393, 46393, 46393, 46393, 46393, 46393, 46393, 46393, 46393]])
tensor([0, 0])
Actual labels: tensor([0, 0])
Predicted labels: tensor([0, 0])


# Validation dataset Accuracy

In [85]:

overall_accuracy = []
total_samples = len(validation_dataset)*16
print(total_samples)
for idx, sample in enumerate(validation_dataset):
  # print(len(sample[0]))
  actual_label = sample[0]
  with torch.no_grad():
    predicted_label = model(sample[1])
    rounded_preds = torch.round(predicted_label)
    _, pred_label = torch.max(rounded_preds, dim=1)
    # print('-'*100)
    # print(actual_label)
    # print(pred_label)
    correct = (pred_label == actual_label).float()
    # print(correct)
    # print(torch.sum(correct))
    # print(total_samples)
    overall_accuracy.append(torch.sum(correct))
    # print(torch.sum(correct))

# finally:
# print(overall_accuracy[:30])

overall_accuracy_final = []
for item in overall_accuracy:
  overall_accuracy_final.append(int(item))

print(f"Total validation accuracy is: {(sum(overall_accuracy_final)/total_samples)*100}")


10656
Total validation accuracy is: 87.41554054054053
